<a href="https://colab.research.google.com/github/stevengregori92/Document-Similarity-Keyword-Extraction/blob/main/Document_Similarity_%26_Keyword_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words('indonesian') + list(punctuation)

#Import Dataset

In [ ]:
data = pd.read_csv('kompas.csv')
data.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


#Extract TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(data.teks)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


#TFIDF Similarity -> Document Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [ ]:
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]])

In [ ]:
data.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [ ]:
data.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [ ]:
data.teks[215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

In [ ]:
data.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

#Keyword Extraction

In [ ]:
vocab = tfidf.get_feature_names_out()
vocab[-10:]

array(['zuniga memilih', 'zunnatul', 'zunnatul mafruhah', 'zurich',
       'zurich northholt', 'zw', 'zw suparman', 'zw tim', 'zx',
       'zx diserbu'], dtype=object)

In [ ]:
tfidf_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274212, 365469, 365468, ..., 386379, 436652, 169219])

In [ ]:
vocab[169219]

'ginandjar'

In [ ]:
vocab[274212]

'litbang bem'

In [ ]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

In [ ]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
  matrix = tfidf.transform([doc])
  vocab = tfidf.get_feature_names_out()

  sorted_tfidf = matrix[0].toarray()[0].argsort()
  return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]

In [ ]:
text= '''Liputan6.com, Jakarta Otoritas Jasa Keuangan (OJK) mencatat per April 2023 dana 
yang terhimpun dari pasar modal melalui efek bersifat utang (EBUS), rights issue, dan 
IPO mencapai Rp 84,01 triliun, dengan jumlah emiten baru sebanyak 33 emiten."Penghimpunan 
dana di pasar modal di bulan April masih terjaga tinggi," kata Kepala Eksekutif Pengawas Pasar 
Modal Keuangan Derivatif dan Bursa Karbon OJK, Inarno Djajadi, dalam konferensi pers Hasil 
rapat Dewan Komisioner Bulan April 2023, Jumat (5/5/2023). Adapun per 28 April 2023, OJK 
mencatat terdapat 63 perusahaan yang akan menggelar penawaran umum saham perdana atau Initial 
Public Offering (IPO) dengan nominal Rp 61,71 triliun.

Maka, secara keseluruhan tercatat ada 115 pipeline yang berada dalam penawaran umum dengan 
nilai Rp 135,31 triliun. Disisi lain, Inarno menyebut juga ada 12 penawaran umum terbatas 
(PUT) atau rights issue sebesar Rp 20,56 triliun. Serta, ada 16 penawaran umum EBUS sebesar 
Rp 21,06 triliun.

Kemudian, terdapat 24 rencana penawaran umum berkelanjutan EBUS Tahap I dan II sebesar 
Rp 31,99 triliun. OJK pun berharap total penghimpunan dana di pasar modal bisa tembus di 
kisaran Rp 170 triliun.

Kendati demikian, OJK berhasil mendapatkan 16 penyelenggara Securities Crowdfunding (SCF) 
dari 338 penerbit dengan total 147.142 pemodal hingga 28 APril 2023. Adapun dari angka tersebut 
dana yang berhasil diperoleh dari SCF mencapai Rp 828,58 miliar.

Sebelumnya, Otoritas Jasa Keuangan (OJK) menyebutkan sektor jasa keuangan tetap terjaga 
sehingga berkontribusi untuk kinerja perekonomian nasional di tengah ketidakpastian global.

Dari pasar saham, Kepala Eksekutif Pengawas Pasar Modal, Keuangan Derivatif dan Bursa Karbon 
OJK, Inarno Djajadi menerangkan indeks harga saham gabungan (IHSG) sampai dengan 24 Februari 
2023 tercatat menguat sebesar 0,25 persen secara month to date (mtd). Hal itu terjadi seiring 
investor non-residen yang membukukan inflow sebesar Rp 3,38 triliun.

 “Secara year to date, indeks harga saham gabungan menguat tipis yaitu 0,09 persen dengan inflow 
 investor non-resident sebesar Rp 162,8 miliar,” kata dia dalam Konferensi Pers Rapat Dewan 
 Komisaris Bulanan Februari 2023, Senin (27/2/2023).

Sementara di pasar obligasi, indeks ICBI menguat 0,04 persen mtd atau 1,53 persen year to date 
ke level 350,07. Untuk pasar obligasi korporasi, aliran dana keluar investor non-residen tercatat 
sebesar Rp 84,2 miliar secara mtd dan Rp 177,2 miliar secara ytd.

Di pasar SBN non-residen mencatatkan outflow sebesar Rp 5,82 triliun mtd. Namun secara year 
to date membukukan inflow sebesar Rp 43,88 triliun. Adapun rata-rata yield SBN pada seluruh 
tenor secara mtd naik sebesar 6,20 bps, pun demikian secara year to date masih menguat 12,66 bps.

Nilai aktiva aktiva bersih atau NAB reksa dana tercatat sebesar Rp 59,18 triliun atau menurun 
0,05 persen mtd dengan investor reksa dana membukukan net subscription sebesar Rp 3,96 triliun 
mtd. Secara yts, NAB reksa dana tumbuh 0,85 persen dan tercatat net subscription sebesar Rp 7,88 triliun.

Penghimpunan dana perusahaan melalui pasar modal hingga 24 Februari 2023 tercatat Rp 35,8 triliun 
dengan jumlah emiten baru tercatat 17 emiten.

“Di pipeline masih terdapat 73 rencana penawaran umum dengan nilai sebesar Rp 108,4 triliun, yang 
di antaranya merupakan rencana yang akan dilakukan oleh 45 calon emiten baru,” sebut Inarno.

Sedangkan untuk penggalangan dana pada securities crowdfunding yang merupakan alternatif pendanaan 
bagi UMKM, telah terdapat 16 penyelenggara yang telah mendapatkan izin dari OJK dengan 360 penerbit 
142.474 pemodal dan total dana yang dihimpun sebesar Rp 778,5 miliar.

Tren pertumbuhan jumlah investor terus berlanjut dengan jumlah investor pasar modal mencapai 10,6 
juta investor per 23 Februari 2023. 
'''

In [ ]:
extract_keywords_tfidf(text, tfidf)

['triliun',
 'rp',
 'emiten',
 'pasar modal',
 'date',
 'miliar',
 'penawaran',
 'pasar',
 'year',
 'dana']